In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras import optimizers
from keras.models import load_model
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import sys 
import os
sys.path.append(os.path.abspath("./models/"))

In [3]:
img_rows, img_cols = 28, 28
num_classes = 10
epochs_count = 10
batch_size = 16

In [4]:
d_train = pd.read_csv('data/train.csv')

In [5]:
pixels = d_train.columns.drop('label')
y_column = 'label'

In [6]:
d_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Нормализация

In [7]:
d_train[pixels] = d_train[pixels].applymap(lambda x: x / 255)

## Готовим выборку

In [8]:
train, validation = train_test_split(d_train, test_size=0.1)

In [9]:
mat_train = train[pixels].as_matrix()
x_train = mat_train.reshape(mat_train.shape[0], img_rows, img_cols, 1)
y_train = train[y_column].as_matrix()

mat_val = validation[pixels].as_matrix()
x_val = mat_val.reshape(mat_val.shape[0], img_rows, img_cols, 1)
y_val = validation[y_column].as_matrix()

In [10]:
y_train = y_train.reshape((y_train.shape[0], 1))
y_val = y_val.reshape((y_val.shape[0], 1))

In [11]:
y_train = np_utils.to_categorical(y_train, 10)
y_val = np_utils.to_categorical(y_val, 10)

In [12]:
print('Data shapes:', x_train.shape, y_train.shape)

Data shapes: (37800, 28, 28, 1) (37800, 10)


In [13]:
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input shape:', input_shape)

Input shape: (28, 28, 1)


In [14]:
from cnn2 import generate_net

In [15]:
model, model_name = generate_net(input_shape, num_classes)

In [16]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs_count, batch_size=batch_size)

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 44s 1ms/step - loss: 0.2082 - acc: 0.9340 - val_loss: 0.0781 - val_acc: 0.9750
Epoch 2/10
37800/37800 [==============================] - 36s 941us/step - loss: 0.0735 - acc: 0.9765 - val_loss: 0.0523 - val_acc: 0.9836
Epoch 3/10
37800/37800 [==============================] - 44s 1ms/step - loss: 0.0575 - acc: 0.9822 - val_loss: 0.0507 - val_acc: 0.9831
Epoch 4/10
37800/37800 [==============================] - 44s 1ms/step - loss: 0.0441 - acc: 0.9859 - val_loss: 0.0562 - val_acc: 0.9838
Epoch 5/10
37800/37800 [==============================] - 40s 1ms/step - loss: 0.0397 - acc: 0.9875 - val_loss: 0.0447 - val_acc: 0.9874
Epoch 6/10
37800/37800 [==============================] - 39s 1ms/step - loss: 0.0339 - acc: 0.9892 - val_loss: 0.0452 - val_acc: 0.9864
Epoch 7/10
37800/37800 [==============================] - 35s 921us/step - loss: 0.0280 - acc: 0.9910 - val_loss: 0.0443 - val_

In [17]:
model_name = model_name + '-e' + str(epochs_count) # Добавим к имени количество эпох

In [18]:
model.save('models/model-' + model_name + '-fitted.h5')

In [19]:
# Если используем предобученную нами модель повторно
#del model
#model = load_model('models/model-cnn-e14-fitted.h5')

In [20]:
score = model.evaluate(x_val, y_val,verbose=0)
print('Test score:   ', score[0])
print('Test accuracy:', score[1])

Test score:    0.0369981144018
Test accuracy: 0.98880952381


In [21]:
dtest = pd.read_csv('data/test.csv', sep=',')
mat_test = dtest.as_matrix()
mat_test = mat_test.reshape(dtest.shape[0], img_rows, img_cols, 1)
y_test_predict = model.predict(mat_test)
y_test_predict = np.argmax(y_test_predict,axis=1)

In [22]:
result = pd.DataFrame({'Label': []})

In [23]:
result['Label'] = y_test_predict
result.index += 1 
result.index.name = 'ImageId'
result.to_csv('data/result-' + model_name + '.csv')
result.head()

,Label
ImageId,
1,2
2,0
3,9
4,9
5,3
